In [0]:
# Loading dataset
events = spark.read.csv("/Volumes/workspace/ecommerce/ecommerce_data/2019-Nov.csv", header = True, inferSchema = True)


In [0]:
from delta.tables import DeltaTable

In [0]:
# MERGE for incremental updates

In [0]:
deltaTable = DeltaTable.forPath(spark, "/Volumes/workspace/ecommerce/ecommerce_data/df_nov")
updates = spark.read.csv("/Volumes/workspace/ecommerce/ecommerce_data/2019-Oct.csv", header=True, inferSchema=True)

In [0]:
deltaTable.alias("t").merge(
    updates.alias("s"),
    "t.user_session = s.user_session AND t.event_time = s.event_time"
).whenMatchedUpdateAll() \
 .whenNotMatchedInsertAll() \
 .execute()

DataFrame[num_affected_rows: bigint, num_updated_rows: bigint, num_deleted_rows: bigint, num_inserted_rows: bigint]

In [0]:
# Time travel
v0 = spark.read.format("delta").option("versionAsOf", 0).load("df_nov")
yesterday = spark.read.format("delta") \
    .option("timestampAsOf", "2026-01-12").load("df_nov")

In [0]:
spark.sql("OPTIMIZE delta.`/Volumes/workspace/ecommerce/ecommerce_data/df_nov` ZORDER BY (event_type, user_id)")
spark.sql("VACUUM delta.`/Volumes/workspace/ecommerce/ecommerce_data/df_nov` RETAIN 168 HOURS")

DataFrame[path: string]